In [3]:
(import "math")

(math)

In [2]:
(define (identity x) x)

(define (inc x)
  (+ x 1))

(define (cube x)
  (* x x x))

(define (sum term a next b)
  (if (> a b)
      0
      (+ (term a)
         (sum term (next a) next b))))

### Exercise 1.29
Redefine integral using simpson's rule.

In [3]:
(define (simpson-integral f a b n)
  (define h
    (/ (- b a) n))
  (define (y k)
    (f (+ a (* k h))))
  (define (term x)
    (cond 
     ((or (= x 0) (= x n)) (y x))
     ((= (% x 2) 1) (* 4 (y x)))
     ((= (% x 2) 0) (* 2 (y x)))))
  (* (/ h 3) (sum term 0 inc n)))

In [4]:
(simpson-integral cube 0 1 1000)

1/4

### Exercise 1.30
Define sum performing as iterative procedure

In [5]:
(define (sum-iterative term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (+ (term a) result))))
  (iter a 0))

In [6]:
(define (sum-integers a b)
  (sum-iterative identity a inc b))

(sum-integers 1 5)

15

In [7]:
(define (sum-cubes a b)
  (sum-iterative cube a inc b))

(sum-cubes 1 3)

36

### Exercise 1.31
a. write an analogous procedure called product that return the *product* of the values of a function at points over a given range. Show how to define factorial in terms of *product*. 
Also use *product* to compute approximations to pi usin the formula  

$\frac{\pi}{4} = \frac{2\cdot 4\cdot 4\cdot 6 \cdot 6 \cdot 8\cdot\cdot\cdot}{3\cdot 3\cdot 5\cdot 5 \cdot 7 \cdot 7\cdot\cdot\cdot}$


In [8]:
(define (product term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (* (term a) result))))
  (iter a 1))

(define (multiply-integers a b)
  (product * a inc b))

(multiply-integers 1 5)

120

In [9]:
(define (factorial x)
  (product * 1 inc x))

(factorial 5)

120

In [10]:
(define (approx-pi n)
  (define (term x)
    (/ (* 2 (+ (// x 2) 1)) (+ 1 (* 2 (+ 1 (// (- x 1) 2))))))
  (* 4.0
     (product term 1 inc n)))

In [11]:
(approx-pi 30)

3.1910574333896466

b. write recursive process version of *product*.

In [12]:
(define (product-recursive term a next b)
  (if (> a b)
      1
      (* (term a)
         (product-recursive term (next a) next b))))

In [13]:
(define (approx-pi-recursive n)
  (define (term x)
    (/ (* 2 (+ (// x 2) 1)) (+ 1 (* 2 (+ 1 (// (- x 1) 2))))))
  (* 4.0
     (product-recursive term 1 inc n)))

In [14]:
(approx-pi-recursive 30)

3.1910574333896466

### Exercise 1.32
a. write *accumulate* function
b. write recursive and iterative of it

In [15]:
(define (accumulate-recursive combiner null-value term a next b)
  (if (> a b)
      null-value
      (combiner (term a)
                (accumulate-recursive combiner null-value term (next a) next b))))

(define (accumulate-iterative combiner null-value term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (combiner (term a) result))))
  (iter a null-value))

(define accumulate accumulate-iterative)

In [16]:
(define (sum-acc term a next b)
  (accumulate + 0 term a next b))
(define (product-acc term a next b)
  (accumulate * 1 term a next b))

### Exercise 1.33
write filtered-accumulate

In [17]:
(define (filtered-accumulate combiner null-value term a next b filter)
  (define (iter a result)
    (cond ((> a b) result)
          ((filter a) (iter (next a) (combiner (term a) result)))
          (else (iter (next a) result))))
  (iter a null-value))

In [18]:
(define (even x)
  (= (% x 2) 0))

In [19]:
(define (sum-even a b)
  (filtered-accumulate + 0 identity a inc b even))

In [20]:
(sum-even 1 4)

6

## 1.3.2 Constructing Procedures Using *lambda*

### Exercise 1.34
``` scheme
(define (f g) (g 2))
```
When call `(f f)`, it evaluates `(f 2)` and it evaluated as `(2 2)`.
It throws an error because 2 is not a procedure.

# 1.3.3 Producers as General Methods

In [12]:
(define (close-enough? x y) (< (abs (- x y)) 0.001))
(define (negative? x) (< x 0))
(define (positive? x) (> x 0))
(define (average x y) (/ (+ x y) 2))

In [22]:
(define (search f neg-point pos-point)
  (let ((midpoint (average neg-point pos-point)))
    (if (close-enough? neg-point pos-point)
        midpoint
        (let ((test-value (f midpoint)))
          (cond ((positive? test-value)
                 (search f neg-point midpoint))
                ((negative? test-value)
                 (search f midpoint pos-point))
                (else midpoint))))))

In [23]:
(define (half-interval-method f a b)
  (let ((a-value (f a))
        (b-value (f b)))
    (cond ((and (negative? a-value) (positive? b-value))
           (search f a b))
          ((and (positive? a-value) (negative? b-value))
           (search f b a))
          (else (error "Values are not of opposite sign" a b)))))

In [24]:
(half-interval-method (lambda (x) (- (* x x x) (* 2 x) 3)) 1.0 2.0)

1.89306640625

## Fixed points of functions

In [32]:
(define tolerance 0.00001)
(define (fixed-point f first-guess)
    (define (close-enough? v1 v2) (< (abs (- v1 v2)) tolerance))
    (define (try guess)
      (let ((next (f guess)))
        (if (close-enough? guess next)
            next
            (try next))))
  (try first-guess))

In [33]:
(define (sq x)
  (fixed-point (lambda (y) (average y (/ x y))) 1.0))

In [34]:
(sq 4)

1.0